In [3]:
import os
! apt-get update -qq > /dev/null   
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! pip install nlu  pyspark==2.4.7 > /dev/null   
!pip install flask_ngrok
import nlu

In [14]:
API = "http://ec2-65-2-29-164.ap-south-1.compute.amazonaws.com:5000/api/"

In [21]:
from flask import Flask,request,jsonify
from flask_ngrok import run_with_ngrok
from sklearn.metrics.pairwise import cosine_similarity

import requests

app = Flask(__name__)
run_with_ngrok(app)

multi_pipe = None
col_names = ['sentence_embedding_bert','sentence_embedding_electra', 'sentence_embedding_use']

@app.route("/load-api")
def loadAPI():
    try:
        global multi_pipe
        multi_pipe = nlu.load('use en.embed_sentence.electra embed_sentence.bert')
        return {"status" : 200},200
    except:
        return {"status" : 500},500

@app.route("/generate-predictions",methods = ['GET', 'POST'])
def genearatePredictions():

    title = request.form['title']
    
    if title and multi_pipe:
        predictions = multi_pipe.predict(title,get_embeddings=True).iloc[0]
        
        response = {}

        for col_name in col_names:
            response[col_name] = list(predictions[col_name][0])
        
        return { "predictions" : response ,"status" : 201},201
    else:
        return { "error" :"ERROR" ,"status" : 500}, 500

@app.route("/get-similar-questions",methods=['GET'])
def getSimilarQuestions():
    data = requests.get(API+"discussion/predictions").json()
    embeddings = request.form['predictions']

    if data['status'] == 200 and embeddings:
        predictions = data['predictions']
        print(predictions)
        print(embeddings)
        for i in range(len(predictions)):
            similarity = 0
            for e_col in col_names:
                similarity += cosine_similarity(predictions[i][e_col],embeddings[e_col]) 

            predictions[i]['similarity_score'] = similarity/len(col_names)
        
        return {"similarQuestions" : sorted(predictions,key = lambda x : x['similarity_score'],reverse=True)[:3] , "status" : 201} , 201
    else:
        return { "status" : 500,"error" :"ERROR"}, 500

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7fe6-34-125-96-243.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sent_electra_small_uncased download started this may take some time.
Approximate size to download 48.7 MB
[OK!]
sent_small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]

127.0.0.1 - - [26/Nov/2021 11:54:26] "GET /load-api HTTP/1.1" 200 -


127.0.0.1 - - [26/Nov/2021 11:54:37] "POST /generate-predictions HTTP/1.1" 201 -
[2021-11-26 11:54:48,643] ERROR in app: Exception on /get-similar-questions [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipy

[{'predictions': {'sentence_embedding_bert': [], 'sentence_embedding_electra': [], 'sentence_embedding_use': []}, '_id': '615576871a0240290c55d762'}, {'predictions': {'sentence_embedding_bert': [], 'sentence_embedding_electra': [], 'sentence_embedding_use': []}, '_id': '615576881a0240290c55d763'}, {'predictions': {'sentence_embedding_bert': [], 'sentence_embedding_electra': [], 'sentence_embedding_use': []}, '_id': '615576891a0240290c55d764'}, {'predictions': {'sentence_embedding_bert': [], 'sentence_embedding_electra': [], 'sentence_embedding_use': []}, '_id': '61568f22814dd205d80c44b3'}, {'predictions': {'sentence_embedding_bert': [], 'sentence_embedding_electra': [], 'sentence_embedding_use': []}, '_id': '615739e3a42081563bd0e732'}, {'predictions': {'sentence_embedding_bert': [], 'sentence_embedding_electra': [], 'sentence_embedding_use': []}, '_id': '61573fa0fd002656ea5c48ef'}, {'predictions': {'sentence_embedding_bert': [], 'sentence_embedding_electra': [], 'sentence_embedding_use